In [1]:
import pandas as pd  # Importa pandas.
import sqlite3  # Importa sqlite3.
from itertools import combinations  # Importa 'combinations'.
import csv  # Importa csv

In [28]:
con_exc = sqlite3.connect('/content/cagedexc.db')

## Funções

In [3]:
def contar_por_grupo(conexao, tabela, variaveis):
    # Esta função conta ocorrências únicas de combinações de variáveis.

    colunas = ", ".join(variaveis)  # Formata variáveis para SQL.

    sql = f"""
        SELECT {colunas}, COUNT(*) as admissoes
        FROM {tabela}
        GROUP BY {colunas}
    """  # Constrói a consulta SQL de contagem.

    return pd.read_sql(sql, conexao)  # Executa a consulta e retorna DataFrame.

In [4]:
def contar_por_grupo_demissoes(conexao, tabela, variaveis):
    # Esta função conta ocorrências únicas de combinações de variáveis.

    colunas = ", ".join(variaveis)  # Formata variáveis para SQL.

    sql = f"""
        SELECT {colunas}, COUNT(*) as demissoes
        FROM {tabela}
        GROUP BY {colunas}
    """  # Constrói a consulta SQL de contagem.

    return pd.read_sql(sql, conexao)  # Executa a consulta e retorna DataFrame.

In [5]:
def contagem_total_admissoes(conexao,tabela):
    # Esta função conta o número total de registros na tabela.

    contagem = pd.read_sql(f"SELECT count(*) FROM {tabela} WHERE tipomovimentação LIKE '%Admissao%';", conexao) # Conta total de admissões.

    contagem = contagem.rename(columns={"count(*)": 'admissoes'}) # Renomeia coluna.

    valor = contagem.iloc[0, 0] # Extrai o valor da contagem.

    dados = pd.DataFrame([{
        'uf': "",
        'sexo': "",
        'raçacor': "",
        'faixa_idade': "",
        'graudeinstrução': "",
        'seção': "",
        "admissoes": valor
    }]) # Cria DataFrame padronizado.

    return dados # Retorna o DataFrame.

In [6]:
def contagem_total_demissoes(conexao,tabela):
    # Esta função conta o número total de registros na tabela.

    contagem = pd.read_sql(f"SELECT count(*) FROM {tabela} WHERE tipomovimentação NOT LIKE '%Admissao%';", conexao) # Conta total de demissões.

    contagem = contagem.rename(columns={"count(*)": 'demissoes'}) # Renomeia coluna.

    valor = contagem.iloc[0, 0] # Extrai o valor da contagem.

    dados = pd.DataFrame([{
        'uf': "",
        'sexo': "",
        'raçacor': "",
        'faixa_idade': "",
        'graudeinstrução': "",
        'seção': "",
        "demissoes": valor
    }]) # Cria DataFrame padronizado.

    return dados # Retorna o DataFrame.

In [7]:
def tabela_contagem(n,tabela,variaveis,mes):
    todas_combinacoes = []  # Inicializa lista de combinações.

    for r in range(1, len(variaveis) + 1):  # Itera sobre tamanhos de combinação.
        todas_combinacoes.extend(combinations(variaveis, r))  # Gera e adiciona combinações.

    todas_combinacoes = [list(comb) for comb in todas_combinacoes]  # Converte tuplas em listas.
    df_start = contagem_total_admissoes(con_exc,tabela)  # Contagem total de admissões.

    for comb in todas_combinacoes:
      df = contar_por_grupo(con_exc,tabela,comb)  # Conta por grupo.
      df_start = pd.concat([df_start,df],ignore_index=True)  # Concatena DataFrames.

    df_start.insert(0,'mes',mes)  # Insere coluna 'mes'.
    return df_start  # Retorna o DataFrame final.

In [8]:
def tabela_contagem_demissoes(n,tabela,variaveis,mes):
    todas_combinacoes = []  # Inicializa lista de combinações.

    for r in range(1, len(variaveis) + 1):  # Itera sobre tamanhos de combinação.
        todas_combinacoes.extend(combinations(variaveis, r))  # Gera e adiciona combinações.

    todas_combinacoes = [list(comb) for comb in todas_combinacoes]  # Converte tuplas em listas.
    df_start = contagem_total_demissoes(con_exc,tabela)  # Contagem total de demissões.

    for comb in todas_combinacoes:
      df = contar_por_grupo_demissoes(con_exc,tabela,comb)  # Conta por grupo de demissões.
      df_start = pd.concat([df_start,df],ignore_index=True)  # Concatena DataFrames.

    df_start.insert(0,'mes',mes)  # Insere coluna 'mes'.
    return df_start  # Retorna o DataFrame final.

## Executando planilha de admissões

In [9]:
chaves = ['uf', 'sexo', 'raçacor', 'faixa_idade', 'graudeinstrução', 'seção']  # Define lista de chaves.

In [10]:
dic_dfs = {}  # Inicializa dicionário de DataFrames.

dic = {
    "cagedexc202301": "janeiro",
    "cagedexc202302": "fevereiro",
    "cagedexc202303": "março",
    "cagedexc202304": "abril",
    "cagedexc202305": "maio",
    "cagedexc202306": "junho",
    "cagedexc202307": "julho",
    "cagedexc202308": "agosto",
    "cagedexc202309": "setembro",
    "cagedexc202310": "outubro",
    "cagedexc202311": "novembro",
    "cagedexc202312": "dezembro"
}  # Dicionário mapeando tabelas para nomes de meses.

In [11]:
for chave, valor in dic.items():
    dic_dfs[valor] = tabela_contagem("",chave,chaves,valor).fillna("") # Popula dic_dfs com contagens de admissões.
    print(f"Mes de {valor} executado") # Imprime progresso.

df_final = dic_dfs['janeiro'] # Inicializa df_final com dados de janeiro.
for chave, valor in dic_dfs.items():
    if chave != 'janeiro':
        df_final = pd.concat([df_final,valor],ignore_index=True) # Concatena os demais meses.

df_final # Exibe o DataFrame final.

Mes de janeiro executado
Mes de fevereiro executado
Mes de março executado
Mes de abril executado
Mes de maio executado
Mes de junho executado
Mes de julho executado
Mes de agosto executado
Mes de setembro executado
Mes de outubro executado
Mes de novembro executado
Mes de dezembro executado


,mes,uf,sexo,raçacor,faixa_idade,graudeinstrução,seção,admissoes
0,janeiro,,,,,,,3900
1,janeiro,99,,,,,,35
2,janeiro,AC,,,,,,18
3,janeiro,AL,,,,,,76
4,janeiro,AM,,,,,,168
...,...,...,...,...,...,...,...,...
525864,dezembro,TO,Masculino,Parda,25 a 39,6.A.9.FUND,"Agricultura, Pecuária, Produção Florestal, Pes...",1
525865,dezembro,TO,Masculino,Parda,25 a 39,MEDIO COMPL,"Atividades Profissionais, Científicas e Técnicas",1
525866,dezembro,TO,Masculino,Parda,25 a 39,MEDIO COMPL,"Comércio, Reparação de Veículos Automotores e ...",1
525867,dezembro,TO,Masculino,Parda,40 a 59,MEDIO COMPL,Indústrias de Transformação,1


In [12]:
ufs = ['','CE','BA','PE'] # Lista de UFs.

In [13]:
tabela_filtrada = df_final.query("uf in @ufs") # Filtra o DataFrame por UFs.

In [14]:
#tabela_filtrada.to_csv("exc_admissoes.csv",sep = ';',index=False) # Salva o DataFrame filtrado em CSV.

## Executando planilha de demissões

In [15]:
dic_dfs2 = {} # Inicializa um dicionário vazio para armazenar DataFrames.

In [16]:
for chave, valor in dic.items():
    dic_dfs2[valor] = tabela_contagem_demissoes("",chave,chaves,valor).fillna("") # Popula dic_dfs2 com contagens de demissões.
    print(f"Mes de {valor} executado") # Imprime progresso.

df_final2 = dic_dfs2['janeiro'] # Inicializa df_final2 com dados de janeiro.
for chave, df in dic_dfs2.items(): # Itera sobre o dicionário original 'dic_dfs'.
    if chave != 'janeiro':
        df_final2 = pd.concat([df_final2,df],ignore_index=True) # Concatena os DataFrames.

df_final2 # Exibe o DataFrame final.

Mes de janeiro executado
Mes de fevereiro executado
Mes de março executado
Mes de abril executado
Mes de maio executado
Mes de junho executado
Mes de julho executado
Mes de agosto executado
Mes de setembro executado
Mes de outubro executado
Mes de novembro executado
Mes de dezembro executado


,mes,uf,sexo,raçacor,faixa_idade,graudeinstrução,seção,demissoes
0,janeiro,,,,,,,4000
1,janeiro,99,,,,,,35
2,janeiro,AC,,,,,,18
3,janeiro,AL,,,,,,76
4,janeiro,AM,,,,,,168
...,...,...,...,...,...,...,...,...
525864,dezembro,TO,Masculino,Parda,25 a 39,6.A.9.FUND,"Agricultura, Pecuária, Produção Florestal, Pes...",1
525865,dezembro,TO,Masculino,Parda,25 a 39,MEDIO COMPL,"Atividades Profissionais, Científicas e Técnicas",1
525866,dezembro,TO,Masculino,Parda,25 a 39,MEDIO COMPL,"Comércio, Reparação de Veículos Automotores e ...",1
525867,dezembro,TO,Masculino,Parda,40 a 59,MEDIO COMPL,Indústrias de Transformação,1


In [17]:
tabela_filtrada2 = df_final2.query("uf in @ufs") # Filtra o DataFrame por UFs.

In [18]:
tabela_filtrada2

,mes,uf,sexo,raçacor,faixa_idade,graudeinstrução,seção,demissoes
0,janeiro,,,,,,,4000
6,janeiro,BA,,,,,,535
7,janeiro,CE,,,,,,199
17,janeiro,PE,,,,,,201
29,janeiro,,Feminino,,,,,3707
...,...,...,...,...,...,...,...,...
524373,dezembro,PE,Masculino,Parda,60 ou mais,SUP.COMP,"Transporte, Armazenagem e Correio",1
524374,dezembro,PE,Masculino,Preta,18 a 24,SUP.INCOMP,Educação,1
524375,dezembro,PE,Masculino,Preta,25 a 39,6.A.9.FUND,Atividades Administrativas e Serviços Compleme...,1
524376,dezembro,PE,Masculino,Preta,25 a 39,MEDIO COMPL,Saúde Humana e Serviços Sociais,1


In [20]:
tabela_filtrada2.to_csv("exc_demissoes.csv",sep = ';',index=False) # Salva o DataFrame filtrado em CSV.